In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/MyDrive/Cogni_AI/Hackathon_2")

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
data = pd.read_csv("hate_speech_train.csv")

In [ ]:
data.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,1,2,0,1,RT @HENNERGIZED: &#128557;&#128557;&#128557;&#...
1,3,0,3,0,1,We are back bitches! @vnpacheco21 @xoxoclaire_...
2,3,0,3,0,1,"RT @SuperrrrMcNasty: Lmfao , this bitch was gi..."
3,3,0,3,0,1,@bradley_eckman don't say shit like that lil n...
4,3,1,2,0,1,Young jezzy dat nigguh


In [ ]:
data.shape

(19826, 6)

In [ ]:
df = pd.DataFrame(data = data[['tweet','class']], columns = ['tweet','class'])

In [ ]:
df.head()

,tweet,class
0,RT @HENNERGIZED: &#128557;&#128557;&#128557;&#...,1
1,We are back bitches! @vnpacheco21 @xoxoclaire_...,1
2,"RT @SuperrrrMcNasty: Lmfao , this bitch was gi...",1
3,@bradley_eckman don't say shit like that lil n...,1
4,Young jezzy dat nigguh,1


In [ ]:
df.shape

(19826, 2)

In [ ]:
df.tweet.values

array(['RT @HENNERGIZED: &#128557;&#128557;&#128557;&#8220;@StopBeingSober: Yall say "Why you dating lil girls" like mature hoes just on a rampage outside.&#8221;',
       'We are back bitches! @vnpacheco21 @xoxoclaire_ @treyhunter_ @Medgmon @delaney_jade @sean_steezy @alexistiefa',
       'RT @SuperrrrMcNasty: Lmfao , this bitch was giving head in the back of a classroom she better go tf ahead !',
       ...,
       'RT @FunnyOGtweets: When rednecks fool around with the aux cord http://t.co/Tyzuro7qR5',
       "@PaulMoon8 @yurista4life @justinmiculka01 you fuckin Yankees I'll be there in 2 months I'll meet you pussy",
       "RT @liyyy_: yall females really be cool with cuffin hoe ass niggas &#128514; nope. couldn't be me."],
      dtype=object)

In [ ]:
df_x = df['tweet']
df_y = df['class']

In [ ]:
# to list
tweets = df.tweet.values
#to lower
tweets = [i.lower() for i in tweets]
# removing handles with @
tweets_no_handle = [  re.sub("@\w+",'',sent) for sent in tweets]
# removing hashtag
tweets_clean = [re.sub("#\w+",'',tweet) for tweet in tweets_no_handle]

In [ ]:
tweets_clean

In [ ]:
df_x = pd.DataFrame(data = tweets_clean, columns=['tweet'])
df_2 = df.copy()
df_2['tweet'] = df_x['tweet'] 
df_2.head()

,tweet,class
0,"rt : &;&;&;&;: yall say ""why you dating lil gi...",1
1,we are back bitches!,1
2,"rt : lmfao , this bitch was giving head in the...",1
3,don't say shit like that lil nig .. or i'll g...,1
4,young jezzy dat nigguh,1


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [ ]:
dataframe = pd.DataFrame(columns = ['tweet','class'])
for index,row in df_2.iterrows():
    clean_sentence=[]
    tokens = word_tokenize(row['tweet'])
    for w in tokens:
      w = w.lower()
      if w not in stops:
        pos = pos_tag([w])
        clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1])) 
        clean_sentence.append(clean_word)
    dataframe = dataframe.append({'class':row['class'], 'tweet': ' '.join(clean_sentence)}, ignore_index = True)

In [ ]:
dataframe.head()

,tweet,class
0,rt yall say `` date lil girl '' like mature ho...,1
1,back bitch,1
2,rt lmfao bitch give head back classroom well g...,1
3,n't say shit like lil nig .. 'll give stamp,1
4,young jezzy dat nigguh,1


In [ ]:
dataframe.head()

,tweet,class
0,rt yall say `` dating lil girls '' like mature...,1
1,back bitches,1
2,rt lmfao bitch giving head back classroom bett...,1
3,n't say shit like lil nig .. 'll give stamp,1
4,young jezzy dat nigguh,1


In [ ]:
def get_simple_pos(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('N'):
    return wordnet.NOUN
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [ ]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
dataframe.columns = ['tweet', 'classes' ]

In [ ]:
from sklearn.model_selection import train_test_split
X = dataframe.tweet.values
Y = dataframe.classes
Y = Y.astype('int')

x_train, x_test, y_train, y_test = train_test_split(X,Y)


In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(14869,)
(4957,)
(14869,)
(4957,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
my_vectorizer = vectorizer.fit_transform(x_train)
vocab = vectorizer.vocabulary_

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(my_vectorizer,y_train)
my_test_data = vectorizer.transform(x_test)
my_pred = clf.predict(my_test_data)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB()
nb_clf.fit(my_vectorizer,y_train)
my_test_data = vectorizer.transform(x_test)
my_pred = nb_clf.predict(my_test_data)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(my_pred, y_test)

0.7738551543272141

In [ ]:
clf.predict(vectorizer.transform(['I like this']))

array([1])

In [ ]:
df_pred = pd.DataFrame(data =my_pred)
df_pred.value_counts()

1    4944
2      13
dtype: int64

In [ ]:
dataframe['classes'].value_counts()

1    15350
2     3335
0     1141
Name: classes, dtype: int64